In [ ]:
import torch
import torch.nn as nn
from pathlib import Path
import numpy as np
import math

import torchvision
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F
from torch.autograd import Variable

import keras

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

import tensorflow as tf
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16970420797927591682
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12808750515571645035
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2537539651441067596
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7907478733
locality {
  bus_id: 1
  links {
  }
}
incarnation: 146353078679668634
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
[[22. 28.]
 [49. 64.]]


Using TensorFlow backend.


In [ ]:
DATA_ROOT = Path("../../data/labeled_letters")

fnames = list(DATA_ROOT.glob("*.png"))

In [ ]:
def _is_pil_image(img):
    return isinstance(img, Image.Image)

def auto_crop(img):
    if not _is_pil_image(img):
        raise TypeError('img should be PIL Image. Got {}'.format(type(img)))
        
    return img.crop(img.getbbox())

def fit_box(img, size=40, interpolation=Image.NEAREST):
    if not _is_pil_image(img):
        raise TypeError('img should be PIL Image. Got {}'.format(type(img)))
    if not isinstance(size, int):
        raise TypeError('Got inappropriate size arg: {}'.format(size))

    w, h = img.size
    if (w <= h and h == size) or (h <= w and w == size):
        return img
    
    if w > h:
        ow, oh = size, size * h // w
    else:
        oh, ow = size, size * w // h
        
    img = img.resize((ow, oh), interpolation)
    new_im = Image.new("1", (size, size))
    new_im.paste(img, ((size-ow)//2, (size-oh)//2))
    return new_im

def rotate(img, angle, resample=False, expand=False, center=None):
    if not _is_pil_image(img):
        raise TypeError('img should be PIL Image. Got {}'.format(type(img)))

    return img.rotate(angle, resample, expand, center)

In [ ]:
transform = transforms.Compose([
    transforms.Lambda(auto_crop),
    transforms.Lambda(lambda img: fit_box(img, 40)),
    transforms.RandomRotation(360, expand=True),
    transforms.Lambda(lambda img: fit_box(img, math.ceil(math.sqrt(2)*40)))
    #transforms.Resize(40, Image.NEAREST),
        #transforms.ToTensor(),
])

In [ ]:
def transform(Image.open(fnames[345]))

In [ ]:
def load_image(f):
    img = Image.open(f)
    img.load()
    return img

images = [load_image(f) for f in fnames]